In [1]:
import envPressureSubsEdges
import datetime
import numpy as np
import random
from scipy.special import softmax
from scipy import spatial
import pickle
from tensorboardX import SummaryWriter
from numpy import dot
from numpy.linalg import norm
import torch as th
from Net008_inference import *

In [2]:
scale =9
probDeep = 0.0
predSteps = 4
EPSILON = 0.0


In [3]:
environ = envPressureSubsEdges.CornicheEnv(macrostep=10,predefined=False,scale=scale, GUI=True)

Could not connect to TraCI server at localhost:55211 [Errno 111] Connection refused
 Retrying in 1 seconds


In [4]:
tls = len(environ.action_space)

In [ ]:
obs = environ.reset()
cars,phaseobs = obs

In [ ]:
with open("amatrix_edges.pkl", 'rb') as f:
    A = pickle.load(f)
    
with open("stoppedEdges.pkl", 'rb') as f:
    stoppedEdges = pickle.load(f)
    
indices = {c:i for i,c in enumerate(list(A.columns))}

In [ ]:
name = 'MaxPressure_%3.2f_greedy_scale9' % EPSILON

writer = SummaryWriter(comment="Evaluation_%s" % name)

In [ ]:
nids = th.LongTensor([x for x in range(g.number_of_nodes())])

def PredictOBS(obs):
    entered = torch.zeros((g.number_of_nodes(),1)).cuda().to(DEVICE)
    #consider the red all action
    freeEdges = 1 - np.sum([stoppedEdges[tl][len(stoppedEdges[tl])-1].values.squeeze() for tl in range(tls)], axis=0)
    freeEdges = th.FloatTensor(freeEdges).unsqueeze(1)        

    predicted,_, entered = net(g, nids.cuda().to(DEVICE), th.FloatTensor(obs).cuda().to(DEVICE).unsqueeze(1), freeEdges.cuda().to(DEVICE), entered)
    for i in range(predSteps-1):
        predicted,_, entered = net(g, nids.cuda().to(DEVICE), predicted, freeEdges.cuda().to(DEVICE), entered)
    
    return predicted

In [ ]:
def tlsPressure(INedges,OUTedges,current,prediction):
    global indices
    
    pIN = th.sum(current[[indices[i] for i in INedges]]).item()
    pOUT = th.sum(current[[indices[i] for i in OUTedges]]).item()
    
    return pIN - pOUT

In [ ]:
def getAllPressures(env,indices,counts):
    tlspressures = []
    
    counts = th.FloatTensor(counts)
    
    for tlsID in env.phases.getTls():
        pressures = []
        for phase in env.phases.getTlsPhases(tlsID):
            pressure = tlsPressure(env.edgesDict[tlsID][phase]["incoming"],env.edgesDict[tlsID][phase]["outgoing"],counts, counts)
            pressures.append(pressure)
        tlspressures.append(pressures)
    return tlspressures

In [ ]:
with open("/home/srizzo/Videos/%s" % name,'w') as fo:
    fo.write("exp_name\tstep\twaitingTime\tqueue\tco2\n")
    for step in range(6*60*24):
        pressures = getAllPressures(environ,indices,cars)
        actions = [ np.argmax(pressures[i]) if random.random() > EPSILON else environ.action_space[i].sample() for i in range(tls)]
        obs,_,_,measures = environ.step(actions)
        cars,phaseobs = obs
        if step%60==0:
            time = str(datetime.timedelta(seconds=step))
            #print("Completed trips at minute %s: %d" % (time,measures['completed_trips']))
            print("%d" % measures['waitingTime'])
        writer.add_scalar("waitingTime", measures['waitingTime'], step)
        writer.add_scalar("queue", measures['queue'], step)
        writer.add_scalar("co2", measures['co2'], step)
        environ.conn.gui.screenshot('View #0','/home/srizzo/Videos/screenshots/%s_%d.png' % (name,step))
        fo.write("%s\t%d\t%d\t%d\t%2f\n" % (name,step,measures['waitingTime'],measures['queue'],measures['co2']))

0
991
9137
12114
12117
36090
25394
71380
134752
225481
431842
810186
1319726
1969302
2846812
3919538
5367777
7003511
8936426
11083277
13326126
15649231
17986359
20399206
22866582
25117530
27670374
30267673
32883142
35508733
38165547
40834944
43460365
46177062
48913664
51686287
53953656
56786532
59639821
62540990
65471516
68395893
71227323
74205054
77222527
80282104
83370170
86494624
89727808
92975372
96247347
99547715
102902011
106307397
109686988


In [ ]:
pressures[0]

In [ ]:
pressures

In [ ]:
[indices[i] for i in [0,2,3]]

In [ ]:
random.random()

In [ ]:
stoppedEdges